# Pré processamento dos dados
## Importar bibliotecas e dfs

In [2]:
import pandas as pd
from unidecode import unidecode
import re

# definiçao das colunas vindas do github do case
import yaml
with open("confs.yaml") as f:
    confs = yaml.safe_load(f)

In [3]:
allowed_cols = confs["training_set_schema"] 
df = pd.read_parquet('train.parquet')[allowed_cols]

df.head(5)


,user_input,razaosocial,nome_fantasia,uf,identificador_matriz_filial,data_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,capital_social,porte,cnae_fiscal,descricao_cnae,nome_pais,nome_natureza_juridica,nome_motivo,nome_qualificacao,cnpj
50222,MAGAZINE L,MAGAZINE LUIZA S/A,MAGAZINE LUIZA,SP,filial,0,None,20050317,1.380216e+10,demais,4713004,"Lojas de departamentos ou magazines, exceto lo...",None,Sociedade Anônima Aberta,SEM MOTIVO,Diretor,47960950025388
99431,PNEUS GP,GP PNEUS LTDA,GP PNEUS,PR,filial,20221209,None,20221209,1.156500e+08,demais,4530705,Comércio a varejo de pneumáticos e câmaras-de-ar,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,46378127006695
240152,SANTA CRUZ DISTRIBUIDORA,DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA,SANTA CRUZ,RS,filial,20051103,None,19970624,6.722510e+08,demais,4644301,Comércio atacadista de medicamentos e drogas d...,None,Sociedade Empresária Limitada,SEM MOTIVO,Administrador,61940292004710
207820,DROGALL,DROGAL FARMACEUTICA LTDA,DROGAL JAGUARIUNA,SP,filial,20220613,None,20220613,5.000000e+06,demais,4771701,"Comércio varejista de produtos farmacêuticos, ...",None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,54375647029451
68386,ESTAPAR BRASIL LTDA,"ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVI...",ESTAPAR,ES,filial,20240604,None,20240604,6.456299e+08,demais,5223100,Estacionamento de veículos,None,Sociedade Anônima Aberta,SEM MOTIVO,Presidente,60537263145644


## Tratamento dos dados 
limpeza principal das colunas Razao social, Nome fantasia e input do usuário

In [4]:
separator_pattern = re.compile(r'[-.,;:()\[\]!?@#$%&*_+=/\\|]+')
punctuation_pattern = re.compile(r'[^\w\s]')  
accent_pattern = re.compile(r'[\u0300-\u036f]') 
space_pattern = re.compile(r'\s+')


def remove_juridical_terms(text, additional_stopwords=None):
    """
    Remove termos jurídicos e palavras irrelevantes de nomes de empresas
    
    Args:
        text: Texto pré-processado
        additional_stopwords: Lista de stopwords adicionais para remover
        
    Returns:
        Texto limpo
    """
    # Termos jurídicos básicos (em minúsculas, sem acentos)
    juridical_terms = {
        'sociedade', 'anonima', 'limitada', 'ltda', 'me', 'epp', 'ep', 
        'sa', 's a', 'comercio', 'industria', 'servicos', 'empresa',
        'falido', 'falencia', 'grupo', 'associacao', 'associados',
        'holding', 'cooperativa', 'nacional', 'internacional', 'brasileira', 'solucoes'
    }
    
    # Adicionar stopwords customizadas se fornecidas
    if additional_stopwords:
        juridical_terms |= set(additional_stopwords)
    
    # Dividir o texto em palavras e filtrar
    words = [word for word in text.split() if word not in juridical_terms]
    
    return ' '.join(words).strip()



def simplify_text(text, keep_accents=False, remove_juridical=True, additional_stopwords=None):
    if not isinstance(text, str) or not text.strip():
        return ""
    
    
    # Converter para minúsculas
    text = text.lower()
    
    # Tratamento especial para S.A. e S/A
    text = re.sub(r'\b(s\.?\s*[aàáâã]\.?|s\s*/\s*a)\b', 'sa', text)
    
    # Normalizar separadores - converter para espaços
    text = separator_pattern.sub(' ', text)
    
    # Tratamento de acentos e caracteres especiais
    if not keep_accents:
        text = unidecode(text)  # Remove acentos e normaliza caracteres
        text = punctuation_pattern.sub('', text)  # Remove pontuação
    else:
        # Remove apenas marcas diacríticas mantendo caracteres base
        text = accent_pattern.sub('', text)
        text = punctuation_pattern.sub('', text)  # Remove pontuação
    
    # Normalizar espaços (remove duplos, mantém únicos)
    text = space_pattern.sub(' ', text).strip()
    
    # Substituir termos jurídicos por extenso
    # Usando regex com \b para garantir palavras completas
    text = re.sub(r'\bsa\b', 'sociedade anonima', text)
    text = re.sub(r'\bltda\b', 'limitada', text)
    
    if remove_juridical:
        text = remove_juridical_terms(text, additional_stopwords)
    
    # Normalizar espaços novamente após substituições
    text = space_pattern.sub(' ', text).strip()
    
    return text
def process_dataframe(df, columns, keep_accents=False):
    """
    Process the DataFrame by applying text simplification to specified columns.
    Processa o dataframe aplicando simplificação de texto nas colunas especificadas.
    
    Args:
        df (pd.DataFrame): dataframe para processar.
        columns (list): Lista de colunas para aplicar simplificação.
    
    Returns:
        pd.DataFrame: dataframe processado simplificado.
    """
    df = df.copy() 
    
    for col in columns:
        new_col = f"{col}_clean"
        df[new_col] = df[col].apply(lambda x: simplify_text(x, keep_accents))
            
    df['combined_input'] = df['user_input_clean'] + " - " + df['uf'].astype(str)
    return df

text_columns = ['razaosocial', 'nome_fantasia', 'user_input']

# Processar dataframe
df_clean = process_dataframe(df, text_columns)

# Mostrar resultado
print(df_clean[['user_input', 'uf', 'combined_input']].head())


                      user_input  uf                 combined_input
50222                 MAGAZINE L  SP                magazine l - SP
99431                   PNEUS GP  PR                  pneus gp - PR
240152  SANTA CRUZ DISTRIBUIDORA  RS  santa cruz distribuidora - RS
207820                   DROGALL  SP                   drogall - SP
68386        ESTAPAR BRASIL LTDA  ES            estapar brasil - ES


In [5]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

words_dict = set()
numbers_found = []  

for col in ['razaosocial_clean', 'nome_fantasia_clean']:
    for text in df_clean[col].dropna():
        words = text.split() 
        for word in words:
            if len(word) >= 4:
                # Identificar palavras que contêm dígitos
                if any(char.isdigit() for char in word):
                    numbers_found.append(word)
                # adicionar a versão normalizada
                words_dict.add(word.lower())

print(f"Total unique words: {len(words_dict)}")

# Mantém apenas palavras que não são números puros
filtered_words = {
    word for word in words_dict 
    if not word.isdigit()  # Remove numeros puros
}

# List the pure numbers that were removed
removed_pure_numbers = [word for word in words_dict if word.isdigit()]
print("\nPure Numbers Removed")
print(removed_pure_numbers)

Total unique words: 8631

Pure Numbers Removed
['4369', '2041', '4349', '1000', '9714', '9540', '4393', '12381', '2049', '1850', '4858', '11815', '4685', '4259', '4708', '4142', '2759', '4543', '1433', '3991', '2706', '00189', '4698', '00634', '3350', '3374', '4736', '3627', '4788', '4765', '3341', '3680', '3933', '4587', '4640', '2075', '2033', '4638', '3030', '4511', '4584', '4665', '2119', '3490', '4692', '3872', '2923', '4691', '4531', '2720', '3554', '3858', '3107', '4854', '1405', '4806', '00313', '4494', '4445', '4599', '2654', '4488', '4608', '13603', '3649', '4807', '4246', '1443', '3312', '4440', '7711', '00470', '3944', '1618', '2104', '4809', '0756', '2117', '3225', '12453', '1168', '0761', '5004975', '2922', '2103', '13475', '3129', '1432', '4538', '6450', '5005088', '4326', '5390', '0105', '4311', '0585', '4618', '4202', '10060', '5167', '3592', '0250', '5009', '1331', '4477', '4450', '7936', '12690', '4576', '3361', '4610', '3166', '4606', '4446', '2184', '4318', '0978',

## Criação de um dicionário de suplemento utilizando as palavras contidas em razão social e nome fantasia e salvar em json

In [6]:
import time
from functools import partial
import multiprocessing

# Configuração para funcionar no Jupyter
if __name__ == '__main__' or 'jupyter' in __file__:
    multiprocessing.set_start_method('spawn', force=True)

# Mapa otimizado de teclas adjacentes
KEYBOARD_ADJACENCY = {
    'q': ['w', 'a', 's'], 'w': ['q', 'e', 'a', 's', 'd'], 
    'e': ['w', 'r', 's', 'd', 'f'], 'r': ['e', 't', 'd', 'f', 'g'], 
    't': ['r', 'y', 'f', 'g', 'h'], 'y': ['t', 'u', 'g', 'h', 'j'], 
    'u': ['y', 'i', 'h', 'j', 'k'], 'i': ['u', 'o', 'j', 'k', 'l'], 
    'o': ['i', 'p', 'k', 'l'], 'p': ['o', 'l'],
    'a': ['q', 'w', 's', 'z'], 's': ['a', 'w', 'e', 'd', 'z', 'x', 'c'], 
    'd': ['s', 'e', 'r', 'f', 'x', 'c', 'v'], 'f': ['d', 'r', 't', 'g', 'c', 'v', 'b'], 
    'g': ['f', 't', 'y', 'h', 'v', 'b', 'n'], 'h': ['g', 'y', 'u', 'j', 'b', 'n', 'm'], 
    'j': ['h', 'u', 'i', 'k', 'n', 'm'], 'k': ['j', 'i', 'o', 'l', 'm'], 
    'l': ['k', 'o', 'p'], 'z': ['a', 's', 'x'], 
    'x': ['z', 's', 'd', 'c'], 'c': ['x', 'd', 'f', 'v'], 
    'v': ['c', 'f', 'g', 'b'], 'b': ['v', 'g', 'h', 'n'], 
    'n': ['b', 'h', 'j', 'm'], 'm': ['n', 'j', 'k']
}

# Letras com maior taxa de erro
HIGH_ERROR_LETTERS = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'}

# Limites por tamanho da palavra
VAR_LIMITS = {
    4: 3000, 5: 3000, 6: 3000, 7: 3000, 8: 3000,
    9: 3000, 10: 3000, 11: 3000, 12: 3000, 13: 3000,
    14: 1000, 15: 1000, 16: 1000, 17: 1000, 18: 1000, 19: 1000
}

# Pré-computar valores de adjacência
ADJACENCY_VALUES = {k: set(v) for k, v in KEYBOARD_ADJACENCY.items()}

def generate_typos(word):
    """Gera variações de erros com limites dinâmicos"""
    word_len = len(word)
    max_vars = float('inf')#VAR_LIMITS.get(word_len, 15)
    
    if word_len < 4:
        return set()
        
    variations = set()
    splits = [(word[:i], word[i:]) for i in range(word_len + 1)]
    
    # 1. Omissões (deletes)
    if word_len <= 20:
        for L, R in splits:
            if not R or len(variations) >= max_vars:
                continue
            new_word = L + R[1:]
            if len(new_word) >= 4:
                variations.add(new_word)
    
    # 2. Trocas de letras adjacentes (transposes)
    for L, R in splits:
        if len(R) < 2 or len(variations) >= max_vars:
            continue
        new_word = L + R[1] + R[0] + R[2:]
        variations.add(new_word)
    
    # 3. Substituições (replaces)
    for L, R in splits:
        if not R or len(variations) >= max_vars:
            continue
        if R[0] in HIGH_ERROR_LETTERS and R[0] in ADJACENCY_VALUES:
            for c in ADJACENCY_VALUES[R[0]]:
                if len(variations) >= max_vars:
                    break
                new_word = L + c + R[1:]
                variations.add(new_word)
    
    # 4. Inserções (inserts)
    if word_len <= 20:
        for L, R in splits:
            if len(variations) >= max_vars:
                break
            ref_char = L[-1] if L else R[0] if R else None
            if ref_char and ref_char in HIGH_ERROR_LETTERS and ref_char in ADJACENCY_VALUES:
                for c in ADJACENCY_VALUES[ref_char]:
                    if len(variations) >= max_vars:
                        break
                    new_word = L + c + R
                    if len(new_word) >= 4:
                        variations.add(new_word)
    
    return variations

def process_word(word):
    """Processa uma palavra individualmente"""
    start_time = time.perf_counter()
    variations = generate_typos(word)
    processing_time = time.perf_counter() - start_time
    return word, variations, processing_time

def generate_typos_for_list(word_list, use_parallel=True):
    """Processa uma lista de palavras, com opção paralela"""
    # Para poucas palavras, processa sequencialmente
    if len(word_list) <= 20 or not use_parallel:
        print("Processamento SEQUENCIAL (poucas palavras)")
        results = []
        for i, word in enumerate(word_list):
            result = process_word(word)
            results.append(result)
            print(f"Processada: {i+1}/{len(word_list)} palavras - {word}")
        return results
    
    # Processamento paralelo para listas maiores
    print("Processamento PARALELO")
    total_cores = multiprocessing.cpu_count()
    used_cores = max(1, total_cores - 1)
    batch_size = max(10, len(word_list) // (used_cores * 5))
    
    with multiprocessing.Pool(processes=used_cores) as pool:
        results = []
        for i, res in enumerate(pool.imap(process_word, word_list, chunksize=batch_size)):
            results.append(res)
            if (i+1) % 100 == 0:
                print(f"Processadas: {i+1}/{len(word_list)} palavras")
    
    return results

# Exemplo de uso no Jupyter
if __name__ == "__main__":
    # Sua lista de 4 palavras

    
    print("Iniciando geração de variações...")
    start_total = time.perf_counter()
    
    # Processar a lista (automaticamente usará sequencial para poucas palavras)
    resultados = generate_typos_for_list(filtered_words, use_parallel=False)
    
    total_time = time.perf_counter() - start_total
    
    # Mostrar resultados
    print("\nResultados detalhados:")
    for word, variations, t in resultados:
        print(f"\nPalavra: {word} (tempo: {t:.4f}s)")
        print(f"Variações geradas ({len(variations)}):")
        print(list(variations)[:10])  # Mostra até 10 variações
    
    # Estatísticas finais
    total_variations = sum(len(v) for _, v, _ in resultados)
    print(f"\n{'='*50}")
    print(f"TOTAL DE PALAVRAS PROCESSADAS: {len(filtered_words)}")
    print(f"TEMPO TOTAL: {total_time:.4f} segundos")
    print(f"VARIAÇÕES GERADAS: {total_variations}")
    print(f"MÉDIA: {total_variations/len(filtered_words):.1f} variações por palavra")


Iniciando geração de variações...
Processamento SEQUENCIAL (poucas palavras)
Processada: 1/8227 palavras - valeban
Processada: 2/8227 palavras - belvedere
Processada: 3/8227 palavras - conteudo
Processada: 4/8227 palavras - tancredo
Processada: 5/8227 palavras - forno
Processada: 6/8227 palavras - jaborandi
Processada: 7/8227 palavras - relacoes
Processada: 8/8227 palavras - elevaz
Processada: 9/8227 palavras - bourgeois
Processada: 10/8227 palavras - militares
Processada: 11/8227 palavras - ilopolis
Processada: 12/8227 palavras - iads
Processada: 13/8227 palavras - music
Processada: 14/8227 palavras - esperantina
Processada: 15/8227 palavras - denadai
Processada: 16/8227 palavras - orleans
Processada: 17/8227 palavras - valadao
Processada: 18/8227 palavras - wanderley
Processada: 19/8227 palavras - abam
Processada: 20/8227 palavras - alpha
Processada: 21/8227 palavras - aranha
Processada: 22/8227 palavras - agronomica
Processada: 23/8227 palavras - fullfilment
Processada: 24/8227 pala

In [7]:
import json
with open('variacoes_palavras.json', 'w', encoding='utf-8') as f:
    json.dump(
        {word: list(variations) for word, variations, _ in resultados},
        f,
        ensure_ascii=False,
        indent=2
    )
print("Resultados salvos em 'variacoes_palavras.json'")

Resultados salvos em 'variacoes_palavras.json'


## Exportar df tratado

In [8]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
df_clean.to_parquet('train_clean.parquet', engine='pyarrow', index=False)